In [1]:
from bs4 import BeautifulSoup
import urllib
import re
import json
import time

In [2]:
r = urllib.request.urlopen('http://ohhla.com/YFA_mfdoom.html').read()
soup = BeautifulSoup(r, 'html.parser')

In [3]:
song_list = []
for a in soup.find_all('a'):
    if '.txt">' in str(a):
        lines = str(a).split('">')
        match = re.search(r'href=[\'"]?([^\'" >]+)', lines[0])
        link = match.group(1)
        title = lines[1][:-4]
        song_list.append(link)

In [4]:
song_list[:5]

['anonymous/kmd/mr_hood/crackpot.kmd.txt',
 'anonymous/kmd/mr_hood/who_me.kmd.txt',
 'anonymous/kmd/mr_hood/boogie.kmd.txt',
 'anonymous/kmd/mr_hood/subrocs.kmd.txt',
 'anonymous/kmd/mr_hood/humrush.kmd.txt']

In [6]:
song_text = []
for song in song_list:
    #do this just in case so we don't lose the whole scrape
    try:
        page = urllib.request.urlopen("http://ohhla.com/"+song).read()
        text = BeautifulSoup(page, 'html.parser')
        #can hit error
        try:
            lyrics = str(text.find_all('pre')[0])
        except:
            lyrics = str(text)
        song_text.append(re.sub('<[^>]*>', '', lyrics))
        print('Succesfully scraped: '+song)
    except:
        #can we do a try try excpet? those other text files just don't have pre, I think it would just be song_text.append(re.sub('<[^>]*>', '', lyrics))
        print("Couldn't scrape: "+song)
    time.sleep(.01)

Succesfully scraped: anonymous/kmd/mr_hood/crackpot.kmd.txt
Succesfully scraped: anonymous/kmd/mr_hood/who_me.kmd.txt
Succesfully scraped: anonymous/kmd/mr_hood/boogie.kmd.txt
Succesfully scraped: anonymous/kmd/mr_hood/subrocs.kmd.txt
Succesfully scraped: anonymous/kmd/mr_hood/humrush.kmd.txt
Succesfully scraped: anonymous/kmd/mr_hood/figureof.kmd.txt
Succesfully scraped: anonymous/kmd/mr_hood/b_blues.kmd.txt
Succesfully scraped: anonymous/kmd/mr_hood/n_gritty.kmd.txt
Succesfully scraped: anonymous/kmd/mr_hood/trial_n.kmd.txt
Succesfully scraped: anonymous/kmd/mr_hood/hard_wit.kmd.txt
Succesfully scraped: anonymous/kmd/mr_hood/808_man.kmd.txt
Succesfully scraped: anonymous/kmd/mr_hood/boy_who.kmd.txt
Succesfully scraped: anonymous/kmd/mr_hood/peachfzz.kmd.txt
Succesfully scraped: anonymous/kmd/mr_hood/porkchop.kmd.txt
Succesfully scraped: anonymous/kmd/mr_hood/soulflex.kmd.txt
Succesfully scraped: anonymous/kmd/mr_hood/g_refill.kmd.txt
Succesfully scraped: anonymous/mf_doom/doomsday/do

Succesfully scraped: anonymous/mf_doom/themouse/sofaking.mfd.txt
Succesfully scraped: anonymous/mf_doom/themouse/the_mask.mfd.txt
Succesfully scraped: anonymous/mf_doom/themouse/braksrap.mfd.txt
Succesfully scraped: anonymous/mf_doom/themouse/perfect.mfd.txt
Succesfully scraped: anonymous/mf_doom/themouse/benzie.mfd.txt
Succesfully scraped: anonymous/mf_doom/themouse/oldskool.mfd.txt
Succesfully scraped: anonymous/mf_doom/themouse/athf.mfd.txt
Succesfully scraped: anonymous/mf_doom/themouse/basket.mfd.txt
Succesfully scraped: anonymous/mf_doom/themouse/no_names.mfd.txt
Succesfully scraped: anonymous/mf_doom/themouse/cross.mfd.txt
Succesfully scraped: anonymous/mf_doom/themouse/mince_mt.mfd.txt
Succesfully scraped: anonymous/mf_doom/themouse/vaturine.mfd.txt
Succesfully scraped: anonymous/mf_doom/themouse/spacehos.mfd.txt
Succesfully scraped: anonymous/mf_doom/themouse/badabing.mfd.txt
Succesfully scraped: anonymous/mf_doom/themouse/beef_rap.mfd.txt
Succesfully scraped: anonymous/mf_doo

In [7]:
print(song_text[0:1])

["\nArtist: K.M.D.\nAlbum:  Mr. Hood\nSong:   Mr. Hood at Piocalles Jewelry/Crackpot\nTyped by: OHHLA Webmaster DJ Flash\n\n[Mr. Hood]  Let's enter this jewerly shop.\n[Zev Love]  Ahc'mon Mr. Piocalles man hook it up\n            14K, def bracelet, you can't beat it kid!\n[Piocalles] No I cannot do that.\n            This is not a pawn shop, this is Piocalles Jewelry!\n            Ahh, Mr. Hood, my favorite customer!\n            What can I do for you today?\n[Mr. Hood]  I would like to see some gold rings.\n[Piocalles] Ahh yes we have these stupid phat gold rings,\n            perfect for your masculine hands.\n[Mr. Hood]  Some earrings for my wife.\n[Piocalles] How about these elephant studded diamond earrings,\n            perfect for the woman of your dreams.\n[Mr. Hood]  And a watch, for my cousin.\n[Piocalles] Ahh yes we have a Rolex for just under three-thousand\n            seven-hundred and ninety-two.\n[Mr. Hood]  That is a beautiful watch.\n[Zev Love]  No actually it's two-t

In [40]:
song_data = {}
song_data['Songs'] = []
count = 0
for song in song_text:
    try:
        #sometimes artist is written incorrectly dur
        try:
            artist = re.search(r"Artist: (.*?)\n", song).group(1)
        except:
            artist = re.search(r"Aritst: (.*?)\n", song).group(1)
        album = re.search(r"Album:  (.*?)\n", song).group(1)
        title = re.search(r"Song:   (.*?)\n", song).group(1)
        #sometimes doesn't pull typed right
        typed = re.search(r"Typed by: (.*?)\n", song).group(0)
        lyrics = song.split(typed,1)[1] 
        song_data['Songs'].append({'Title':title,'Album':album, 'Artist':artist, 'Lyrics':lyrics})
    except:
        #again, want to use try excepts for indivdual artist
        song_data['Songs'].append({'Raw':song})
        count += 1

In [41]:
with open('raw_mf_doom.json', 'w') as outfile:
    json.dump(song_data, outfile)

In [2]:
songs = json.load(open('raw_mf_doom.json'))

In [3]:
def clean_lyrics(full_text):
    line_list = []
    #split on /n so that each entry in list is a line
    lines = full_text.split('\n')
    #then make list of words for each line
    for line in lines:
        #keeps all words
        #get rid of all brackets and parenthesees in words (represents someone talking)
        words = re.sub("([\(\[]).*?([\)\]])", "", line)
        #isolate words 
        words = re.findall(r"[\w']+", words)
        if len(words):
            line_list.append(words)
    return line_list

In [ ]:
for song in songs['Songs']:
    lyrics = clean_lyrics(song)